In [22]:
# Initial imports.
import pandas as pd
from datetime import date, timedelta
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
from sklearn import metrics

In [37]:
# Loading weather data
test_df = pd.read_csv("data/test_timeseries.csv")
test_df.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,...,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78,0.0
1,1001,2019-01-02,4.94,100.48,8.65,13.05,11.83,11.74,17.76,9.54,...,13.06,1.02,1.35,0.32,1.03,2.13,3.37,0.56,2.81,NaN
2,1001,2019-01-03,20.74,100.03,8.59,12.12,11.67,11.67,13.74,10.44,...,12.12,1.83,4.23,0.34,3.88,3.41,7.49,0.72,6.77,NaN
3,1001,2019-01-04,16.17,99.47,7.93,11.80,10.38,9.59,18.19,5.44,...,11.72,3.75,5.25,2.34,2.91,6.78,9.61,4.90,4.71,NaN
4,1001,2019-01-05,0.01,100.39,5.32,7.91,4.73,4.71,14.37,3.68,...,7.75,2.32,2.90,1.69,1.21,4.42,5.87,3.15,2.72,NaN


In [38]:
# Loading soil data
soil_df = pd.read_csv("data/soil_data.csv")
soil_df.head()

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1005,31.870670,-85.405456,146,0.0158,0.1868,0.5441,0.2424,0.0106,0.0003,...,72.578804,1.828159,74.406960,3,2,1,1,1,1,1
2,1003,30.659218,-87.746067,52,0.0746,0.4370,0.4415,0.0469,0.0000,0.0000,...,59.843639,2.996914,62.840553,3,2,1,2,1,1,1
3,1007,33.015893,-87.127148,93,0.0144,0.1617,0.3714,0.3493,0.0898,0.0134,...,1.916593,0.008330,1.924924,3,2,1,1,1,1,1
4,1009,33.977448,-86.567246,198,0.0050,0.0872,0.2799,0.3576,0.1477,0.1037,...,1.891909,0.027488,1.919397,3,2,1,1,1,1,1


In [58]:
# Limit the weather data to Texas
texas_weather = test_df.loc[test_df["fips"] >= 48000]
texas_weather = texas_weather.loc[test_df["fips"] <= 48999]

In [59]:
texas_weather = pd.merge(texas_weather, soil_df, on='fips')
texas_weather

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,48001,2019-01-01,0.13,100.79,5.50,6.65,5.21,5.11,10.88,2.71,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
1,48001,2019-01-02,41.48,100.66,5.36,5.57,4.85,4.82,6.63,4.40,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
2,48001,2019-01-03,11.94,100.13,5.19,5.10,4.30,4.25,7.71,0.55,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
3,48001,2019-01-04,0.00,100.20,5.02,5.91,3.84,3.55,12.91,-0.25,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
4,48001,2019-01-05,0.00,100.56,6.34,9.13,7.25,7.09,17.84,3.22,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185669,48507,2020-12-27,0.15,98.98,5.73,13.42,5.62,9.52,23.64,5.57,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
185670,48507,2020-12-28,0.85,99.17,10.15,18.05,14.13,16.09,26.02,10.24,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
185671,48507,2020-12-29,1.56,98.71,10.87,20.17,15.15,17.66,26.36,16.61,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
185672,48507,2020-12-30,8.47,98.39,9.59,16.53,13.12,14.83,24.90,8.39,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3


In [60]:
# Drop redundant weather data
texas_weather = texas_weather.drop(['T2MDEW', 'T2MWET', 'T2M_RANGE', 'WS10M_RANGE', 'WS50M_RANGE'], axis=1)

In [61]:
# Drop unnecessary soil data
texas_weather = texas_weather.drop(['lat', 'lon'], axis=1)

In [62]:
# Set NaN drought scores to the most recent
texas_weather['score'].fillna(method='ffill', inplace=True)

In [63]:
# Account for Seasonality 
# Create column to show drought score 1 year prior
# Need to change week to year when we have full data set
texas_weather['date'] = pd.to_datetime(texas_weather['date'])
time_delta = timedelta(-7)
texas_weather['prev_yr_date'] = texas_weather['date'] + time_delta
texas_weather.head(10)

,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,prev_yr_date
0,48001,2019-01-01,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,...,0.180426,50.434738,3,2,1,1,1,1,1,2018-12-25
1,48001,2019-01-02,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,...,0.180426,50.434738,3,2,1,1,1,1,1,2018-12-26
2,48001,2019-01-03,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,...,0.180426,50.434738,3,2,1,1,1,1,1,2018-12-27
3,48001,2019-01-04,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,...,0.180426,50.434738,3,2,1,1,1,1,1,2018-12-28
4,48001,2019-01-05,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,...,0.180426,50.434738,3,2,1,1,1,1,1,2018-12-29
5,48001,2019-01-06,0.02,100.42,7.95,11.23,17.85,5.05,10.92,4.04,...,0.180426,50.434738,3,2,1,1,1,1,1,2018-12-30
6,48001,2019-01-07,0.02,100.29,10.54,16.43,21.96,12.63,15.84,4.54,...,0.180426,50.434738,3,2,1,1,1,1,1,2018-12-31
7,48001,2019-01-08,0.00,100.82,7.89,12.73,17.70,8.51,12.25,3.39,...,0.180426,50.434738,3,2,1,1,1,1,1,2019-01-01
8,48001,2019-01-09,0.00,101.61,4.91,7.85,12.92,4.27,7.33,3.90,...,0.180426,50.434738,3,2,1,1,1,1,1,2019-01-02
9,48001,2019-01-10,0.00,101.15,4.87,7.41,12.80,2.44,7.22,2.20,...,0.180426,50.434738,3,2,1,1,1,1,1,2019-01-03


In [99]:
day = texas_weather.iloc[[0], [45]]
county = texas_weather.iloc[[0], [1]]
prev_score = texas_weather.loc[texas_weather['fips'] == county]
prev_score

/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: Cannot index with multidimensional key

In [85]:
for i in range(len(texas_weather)):
    day = texas_weather.loc[[i], ['prev_yr_date']]
    #county = texas_weather.loc[[i], ['fips']]
    #prev_score = texas_weather.loc[[('date' == day) & ('fips' == county)]]
    #texas_weather[[i], ['prev_yr_score']] = prev_score

ValueError: PandasArray must be 1-dimensional.

In [57]:
# Drop NaN drought scores
texas_weather.dropna(inplace=True)
texas_weather.columns

Index(['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2M_MAX', 'T2M_MIN',
       'TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS50M', 'WS50M_MAX',
       'WS50M_MIN', 'score', 'elevation', 'slope1', 'slope2', 'slope3',
       'slope4', 'slope5', 'slope6', 'slope7', 'slope8', 'aspectN', 'aspectE',
       'aspectS', 'aspectW', 'aspectUnknown', 'WAT_LAND', 'NVG_LAND',
       'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND', 'CULTIR_LAND',
       'CULT_LAND', 'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6', 'SQ7',
       'prev_yr_date', 'prev_yr_drought'],
      dtype='object')

In [29]:
# Drop NaN drought scores
test_df.dropna(inplace=True)
# Set NaN drought scores to the most recent
# test_df.fillna(method='ffill', inplace=True)
test_df.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,...,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78,0.0
7,1001,2019-01-08,0.05,100.57,8.75,13.15,12.01,11.98,20.04,9.36,...,13.05,2.33,3.40,1.68,1.72,5.13,6.51,3.67,2.85,0.0
14,1001,2019-01-15,0.00,101.07,4.04,3.91,0.94,0.96,9.40,1.02,...,3.91,1.73,2.40,1.36,1.04,3.28,4.13,2.39,1.75,0.0
21,1001,2019-01-22,0.03,101.14,4.96,6.03,3.81,3.08,12.43,-1.63,...,6.08,3.68,4.57,2.49,2.07,6.65,8.56,5.01,3.55,0.0
28,1001,2019-01-29,5.79,100.36,3.69,2.23,-0.41,-1.03,7.33,-5.19,...,2.18,3.10,4.52,1.71,2.81,5.62,6.96,3.60,3.35,0.0


In [30]:
# Define the features set.
X = test_df.copy()
X = X.drop("score", axis=1)
X = X.drop("date", axis=1)
X.head()

,fips,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,1001,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,6.46,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78
7,1001,0.05,100.57,8.75,13.15,12.01,11.98,20.04,9.36,10.68,13.05,2.33,3.40,1.68,1.72,5.13,6.51,3.67,2.85
14,1001,0.00,101.07,4.04,3.91,0.94,0.96,9.40,1.02,8.38,3.91,1.73,2.40,1.36,1.04,3.28,4.13,2.39,1.75
21,1001,0.03,101.14,4.96,6.03,3.81,3.08,12.43,-1.63,14.06,6.08,3.68,4.57,2.49,2.07,6.65,8.56,5.01,3.55
28,1001,5.79,100.36,3.69,2.23,-0.41,-1.03,7.33,-5.19,12.52,2.18,3.10,4.52,1.71,2.81,5.62,6.96,3.60,3.35


In [31]:
# Define the target set.
y = test_df["score"].ravel()
y[:5]

array([0., 0., 0., 0., 0.])

In [32]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [33]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# Create a random forest classifier.
rf_model = RandomForestRegressor(n_estimators=128, random_state=1) 

In [35]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [36]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [41]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
mape = np.mean(np.abs((y_test - predictions) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

Mean Absolute Error (MAE): 0.26875707529915116
Mean Squared Error (MSE): 0.227910793374645
Root Mean Squared Error (RMSE): 0.4774000349545913
Mean Absolute Percentage Error (MAPE): nan
Accuracy: nan


/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [42]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.13035395, 0.04748366, 0.16337887, 0.04441748, 0.03028304,
       0.05333723, 0.05933023, 0.05599217, 0.05070245, 0.06168139,
       0.04967446, 0.02817336, 0.03157874, 0.02868458, 0.03622582,
       0.03070193, 0.03344309, 0.02838346, 0.03617407])